In [1]:
import pandas as pd

In [39]:
HEADERS = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate',
            'DNT': '1',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1'
        }

import requests
from bs4 import BeautifulSoup

metro_district = ['Печерский', 'Шевченковский']
district_dict = {'Печерский': 2.87, 'Шевченковский': 1.74, 'Голосеевский': 1.53, 'Подольский': 1.29, 'Оболонский': 1.22, 'Соломенский': 1.19, 'Днепровский': 1.19, 'Святошинский': 1.18, 'Дарницкий': 1.12, 'Деснянский': 1.00}

district_list = []
area_list = []
rooms_list = []
floor_list = []
near_metro_list = []
gk_list = []
adress_list = []
price_list = []
latit_long = []
repairs = []
for page in range (0, 387):
    SITE = f'https://100realty.ua/realty_search/apartment/rent/p_4600_300000/cur_3/kch_2?page={page}'

    r = requests.get(SITE, headers = HEADERS)

    if r.status_code == 200:
        soup = BeautifulSoup(r.text, 'lxml')
        
        for el in soup.find_all('div', class_ = "object-common-wrapper"): 
            adr = el.find('div', class_ = "object-address").text # адрес
            #print(adr)
            adress_list.append(adr)
#             id_object = adr.find('a').get('href').replace('/object/','') # id
#             link = 'https://100realty.ua/' + adr.find('a').get('href') # ссылка
             # район:
            try:
                object_region = el.find('div', class_ = "object-region").text.split()[-2]
                #print(object_region)
                number_distr = district_dict.get(object_region)
                #print(number_distr)
                district_list.append(number_distr)
            except IndexError as e:
                   district_list.append(None)
                
                # площадь
            object_square = float(el.find('div', class_ = "object-square object-info-item").text.split()[-2])
            area_list.append(object_square)
            
            # комнаты
            rooms_number = int(el.find('div', class_ = "object-rooms object-info-item").text.split()[2])
            rooms_list.append(rooms_number)
                # этаж
            fl = el.find('div', class_ = "object-floors object-info-item")
            if fl is not None:
                floor = fl.find('div', class_ = "value").text.split('/')[0]
                if len(floor) < 3:
                    floor_list.append(int(floor))
                else:
                    floor_list.append(1)
            else:
                floor_list.append(4)
                
               #метро
            try:
                s = el.find('div', class_ = "area").text
                if 'метро' in s:
                    near_metro_list.append(3)
                    #print(1)
                else:
                    if district in metro_district:
                        near_metro_list.append(3)
                    else:
                        if 'метро' in el.find('div', class_='descr').text:
                            near_metro_list.append(2)
                            #print(2)
                        else:
                            near_metro_list.append(1)
                            #print(3)

            except AttributeError as e:
                near_metro_list.append(1)                    
         
            # ЖК
            v = el.find('div', class_ = "object-resident object-info-item") 
            if v is not None:
                living_complex = v.find('div', class_ = "value").text
                gk_list.append(1)
            else:
                gk_list.append(0)
                
            pr = el.find('div', class_ = "cost-field") 
            unformatted_price = pr.find('span').text.replace(' грн./месяц','').replace('*','')
            price = int(unformatted_price.replace(' ',''))
            price_list.append(price)
            
            link2='https://100realty.ua'+(el.find('div', class_='object-address').find('a').get('href'))
            r2 = requests.get(link2, headers=HEADERS)
            if r2.status_code == 200:
                soup2 = BeautifulSoup(r2.text, 'lxml')
                try:
                    lat = soup2.find('div', class_="object-map__content")['data-lat']
                    long = soup2.find('div', class_="object-map__content")['data-lng']
                    latit_long.append((lat, long))

                except TypeError as e:
                    latit_long.append(None)
                a = 0
                build = soup2.find_all('div', class_="object-info-row")
                try:              
                    for text in build:
                       
                        repair = text.find('div', class_="value").text
                        if ('евроремонтом' or 'с ремонтом') in repair:
                            a = 1
                            repairs.append(1)
                                                            
                except AttributeError as e:
                    pass
                if a == 0:
                    repairs.append(0)
    
df = pd.DataFrame({'district': district_list, 'metro': near_metro_list, 'area': area_list, 'rooms': rooms_list, 'floor': floor_list, 'gk': gk_list, 'price': price_list, 'adress': adress_list, 'lat_long': latit_long, 'repair': repairs})

            
df 

,district,metro,area,rooms,floor,gk,price,adress,lat_long,repair
0,1.19,1,85.0,2,4,0,30200,"\nул. Липкивского Василия (Урицкого), 18, Киев\n","(50.4339983, 30.4843094)",0
1,1.22,3,40.0,1,4,0,10900,"\nул. Богатырская, 6а, Киев\n","(50.5201598, 30.4849631)",0
2,1.22,1,40.0,1,4,1,9500,"\nул. Калнишевского Петра (Майорова М.), 8, Ки...","(50.5222776, 30.4663735)",0
3,1.29,3,50.0,2,4,0,25000,"\nул. Боричев Ток, 8, Киев\n","(50.4594295, 30.5215163)",1
4,1.12,1,97.0,3,23,1,65900,"\nул. Днепровская наб., 14б, Киев\n","(50.4017441, 30.614014)",0
...,...,...,...,...,...,...,...,...,...,...
7718,1.19,3,70.0,2,4,0,22000,"\nул. Никольско-Слободская, 3а, Киев\n","(50.4551297, 30.5861282)",0
7719,1.19,1,41.0,1,2,1,14000,"\nул. Регенераторная, 4 корпус 2, Киев\n","(50.4375504, 30.6213166)",0
7720,2.87,1,170.0,4,4,1,68600,"\nДружбы Народов бульв., 14-16, Киев\n","(50.413621, 30.5372004)",0
7721,1.74,3,50.0,2,4,1,27500,"\nул. Златоустовская, 30, Киев\n","(50.4505902, 30.4879963)",0


NameError: name 'df' is not defined

In [41]:
df1 = df.dropna()


In [7]:
df1 = pd.read_csv('rent100riel_coordin_repair_new_dist.csv')
df1.head()

,district,metro,area,rooms,floor,gk,price,adress,lat_long,repair,metro_dist
0,1.19,1,85.0,2,4,0,30200,"\nул. Липкивского Василия (Урицкого), 18, Киев\n","('50.4339983', '30.4843094')",0,3
1,1.22,3,40.0,1,4,0,10900,"\nул. Богатырская, 6а, Киев\n","('50.5201598', '30.4849631')",0,1
2,1.22,1,40.0,1,4,1,9500,"\nул. Калнишевского Петра (Майорова М.), 8, Ки...","('50.5222776', '30.4663735')",0,1
3,1.29,3,50.0,2,4,0,25000,"\nул. Боричев Ток, 8, Киев\n","('50.4594295', '30.5215163')",1,1
4,1.12,1,97.0,3,23,1,65900,"\nул. Днепровская наб., 14б, Киев\n","('50.4017441', '30.614014')",0,1


In [8]:
print(len(df1))

7394


In [9]:
import pickle 
with open(r'C:\Users\Алекс\Documents\github\DS-6\rent flats progect\metro.pickle', 'rb') as f:     #pickle працює в бінарному режимі 
    result = pickle.load(f) 
    


result

,0,1,2
0,Sviatoshyn (Kyiv Metro),50.457500,30.391944
1,Vystavkovyi Tsentr (Kyiv Metro),50.382500,30.477500
2,Khreshchatyk (Kyiv Metro),50.447222,30.522778
3,Dorohozhychi (Kyiv Metro),50.473611,30.449167
4,Syrets (Kyiv Metro),50.476389,30.430833
5,Kontraktova Ploshcha (Kyiv Metro),50.465278,30.516667
6,Zoloti Vorota (Kyiv Metro),50.448333,30.513333
7,Vokzalna (Kyiv Metro),50.441667,30.488056
8,Maidan Nezalezhnosti (Kyiv Metro),50.450000,30.524444
9,Palats Sportu (Kyiv Metro),50.438056,30.520833


In [52]:
name_stat = result[0].tolist()
name_stat

['Sviatoshyn (Kyiv Metro)',
 'Vystavkovyi Tsentr (Kyiv Metro)',
 'Khreshchatyk (Kyiv Metro)',
 'Dorohozhychi (Kyiv Metro)',
 'Syrets (Kyiv Metro)',
 'Kontraktova Ploshcha (Kyiv Metro)',
 'Zoloti Vorota (Kyiv Metro)',
 'Vokzalna (Kyiv Metro)',
 'Maidan Nezalezhnosti (Kyiv Metro)',
 'Palats Sportu (Kyiv Metro)',
 'Chervonyi Khutir (Kyiv Metro)',
 'Beresteiska (Kyiv Metro)',
 'Vyrlytsia (Kyiv Metro)',
 'Pochaina (Kyiv Metro)',
 'Teremky (Kyiv Metro)',
 'Obolon (Kyiv Metro)',
 'Olimpiiska (Kyiv Metro)',
 'Demiivska (Kyiv Metro)',
 'Tarasa Shevchenka (Kyiv Metro)',
 'Darnytsia (Kyiv Metro)',
 'Zhytomyrska (Kyiv Metro)',
 'Lisova (Kyiv Metro)',
 'Vydubychi (Kyiv Metro)',
 'Klovska (Kyiv Metro)',
 'Lukianivska (Kyiv Metro)',
 'Kharkivska (Kyiv Metro)',
 'Pozniaky (Kyiv Metro)',
 'Heroiv Dnipra (Kyiv Metro)',
 'Lvivska Brama (Kyiv Metro)',
 'Osokorky (Kyiv Metro)',
 'Holosiivska (Kyiv Metro)',
 'Boryspilska (Kyiv Metro)',
 'Lybidska (Kyiv Metro)',
 'Telychka (Kyiv Metro)',
 'Ipodrom (Kyiv Metr

In [12]:
lat_metro_list = result[1].tolist()

long_metro_list = result[2].tolist()

# new_list_lat_m = []
# new_list_long_m = []
# for i in range(len(lat_metro_list)):
#     new_list_lat_m.append(lat_metro_list[i])
#     new_list_long_m.append(long_metro_list[i])



In [48]:
list_metro_for_func = []
for i in range(len(lat_metro_list)):
    list_metro_for_func.append([lat_metro_list[i], long_metro_list[i]])


In [54]:
metro_dict = {}
for i in range(len(name_stat)):
    metro_dict[name_stat[i]] = (lat_metro_list[i], long_metro_list[i])
               
metro_dict

{'Sviatoshyn (Kyiv Metro)': (50.4575, 30.391944444444444),
 'Vystavkovyi Tsentr (Kyiv Metro)': (50.3825, 30.4775),
 'Khreshchatyk (Kyiv Metro)': (50.447222222222216, 30.522777777777776),
 'Dorohozhychi (Kyiv Metro)': (50.47361111111111, 30.449166666666667),
 'Syrets (Kyiv Metro)': (50.47638888888889, 30.430833333333336),
 'Kontraktova Ploshcha (Kyiv Metro)': (50.46527777777778, 30.516666666666666),
 'Zoloti Vorota (Kyiv Metro)': (50.44833333333333, 30.513333333333332),
 'Vokzalna (Kyiv Metro)': (50.44166666666666, 30.488055555555558),
 'Maidan Nezalezhnosti (Kyiv Metro)': (50.45, 30.524444444444445),
 'Palats Sportu (Kyiv Metro)': (50.43805555555555, 30.520833333333332),
 'Chervonyi Khutir (Kyiv Metro)': (50.40888888888889, 30.694444444444446),
 'Beresteiska (Kyiv Metro)': (50.45861111111111, 30.419722222222223),
 'Vyrlytsia (Kyiv Metro)': (50.40333333333333, 30.66611111111111),
 'Pochaina (Kyiv Metro)': (50.48694444444445, 30.497777777777777),
 'Teremky (Kyiv Metro)': (50.367044444444

In [64]:
import pickle

with open(r'metro_dict_coord.pickle', 'wb') as f:
    pickle.dump(metro_dict, f)

In [38]:



# col_one_list = pd.to_numeric(df1['lat_long'])
# # l = [list(i) for i in col_one_list]
# # new_list_lat = []
# # new_list_long = []
# # for i in col_one_list:
# #     new_list_lat.append(i[0])
# #     new_list_long.append(i[1])

# col_one_list 

In [179]:
from math import radians, cos, sin, asin, sqrt

def metro_distance(lat1, long1): 
    list_dist = []
    for i in list_metro_for_func:
        lat1 =  radians(lat1)
        long1 = radians(long1)
        lat2 = radians(i[0])
        long2 = radians(i[1])

        dlong =  long2 - long1
        dlat = lat2 - lat1
        g = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlong / 2)**2
        c = 2 * asin(sqrt(g))

        r = 6371   # Radius of earth in kilometers.
        dist = (c * r)

        list_dist.append(dist)
    min_dist = min(list_dist)
    if min_dist < 6:
        dist_coef = 3
    else:
        if min_dist < 12:
            dist_coef = 2
        else:
            dist_coef = 1
    return dist_coef

# l1 = [50.4339983, 50.5201598]
# l2 = [30.4843094, 30.4849631] 

# dist_list1 = []
# for i in range(len(l1)):
#     res = metro_distance(l1[i], l2[i])
#     dist_list1.append(res)
    
# dist_list1

In [65]:
from math import radians, cos, sin, asin


def distance_haversine(point_1: tuple, point_2: tuple):
    d_earth = 2.0 * 6372.8
    lat1, long1 = tuple(radians(c) for c in point_1)
    lat2, long2 = tuple(radians(c) for c in point_2)
    d = sin((lat2 - lat1) / 2.0) ** 2.0 + cos(lat1) * cos(lat2) * sin(
        (long2 - long1) / 2.0) ** 2.0
    return d_earth * asin(d ** 0.5)


def find_nearest(point_1: tuple, points: dict):
    dists = {p: distance_haversine(point_1, points[p]) for p in points}
    name, dist = min(dists.items(), key=lambda d: d[1])
    print(name)
    print(dist)
    return {'name': name, 'distance': dist,
            'dist_coef': 3 if dist <= 1.0 else 2 if dist < 2.0 else 1}


metro_points = {
    'Новокосино': (55.745113, 37.864052),
    'Перово': (55.75098, 37.78422),
    'Ховрино': (55.8777, 37.4877),
    }


point_1 = (50.4594295, 30.5215163)
print(find_nearest(point_1, metro_dict))

Poshtova Ploshcha (Kyiv Metro)
0.24840564091723416
{'name': 'Poshtova Ploshcha (Kyiv Metro)', 'distance': 0.24840564091723416, 'dist_coef': 3}


In [171]:
new_list_long[1]

30.4849631

In [183]:
# df1['metro_dist_new'] = dist_list
df1[df1['metro_dist_new'] == 3]

,district,metro,area,rooms,floor,gk,price,adress,lat_long,repair,metro_dist,metro_dist_new
16,1.74,3,160.0,4,4,1,110000,"\nПобеды просп., 42, Киев\n","[50.4558211, 30.4521936]",0,1,3
31,1.18,1,28.0,1,7,0,8000,"\nул. Симиренко, 28, Киев\n","[50.4077546, 30.4017653]",0,1,3
40,1.19,3,72.0,2,4,0,16500,"\nул. Васильковая, 30, Киев\n","[50.4282552, 30.4529175]",0,2,3
42,1.18,3,53.0,1,4,0,15000,"\nул. Осенняя, 33, Киев\n","[50.4664072, 30.3481097]",0,1,3
44,1.74,3,45.0,1,4,1,28500,"\nул. Шолуденко, 1а, Киев\n","[50.4506775, 30.4755514]",0,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...
7681,1.18,3,110.0,3,4,0,25000,"\nПобеды просп., 73/1, Киев\n","[50.4581426, 30.3999691]",0,1,3
7685,1.74,3,120.0,3,12,0,30000,"\nул. Герцена, 17/25, Киев\n","[50.4701806, 30.4710196]",0,1,3
7690,1.18,1,55.0,2,7,0,10000,"\nул. Ушакова Николая, 12, Киев\n","[50.4580066, 30.3396658]",0,1,3
7698,1.18,3,42.0,1,23,1,9500,"\nПобеды просп., 67б, Киев\n","[50.4579786, 30.4058647]",1,1,3


In [117]:
df1.drop('lat', axis=1, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [118]:
df1.drop('long', axis=1, inplace=True)

In [145]:
list_coordinat = []
for i in range(len(new_list_lat)):
    list_coordinat.append([new_list_lat[i], new_list_long[i]])
list_coordinat 

[[50.4339983, 30.4843094],
 [50.5201598, 30.4849631],
 [50.5222776, 30.4663735],
 [50.4594295, 30.5215163],
 [50.4017441, 30.614014],
 [50.4300933, 30.5216232],
 [50.4530671, 30.4849137],
 [50.409313, 30.6379623],
 [50.4172232, 30.5247632],
 [50.4699102, 30.4804564],
 [50.435735, 30.5096709],
 [50.4456275, 30.4976403],
 [50.4456275, 30.4976403],
 [50.413621, 30.5372004],
 [50.4093352, 30.6396455],
 [50.4240684, 30.5214559],
 [50.4558211, 30.4521936],
 [50.4020697, 30.5264034],
 [50.4099284, 30.6158676],
 [50.430352, 30.517372],
 [50.3843665, 30.4560308],
 [50.4242527, 30.6434615],
 [50.3873111, 30.4713752],
 [50.4348977, 30.5292593],
 [50.4031334, 30.5166686],
 [50.4284434, 30.5462405],
 [50.4392224, 30.5159643],
 [50.4753597, 30.597028],
 [50.4747913, 30.6296633],
 [50.4152075, 30.6368277],
 [50.4077546, 30.4017653],
 [50.436448, 30.5030229],
 [50.441249, 30.533111],
 [50.3946295, 30.6510239],
 [50.4283452, 30.6149146],
 [50.4082691, 30.6228009],
 [50.3861564, 30.4485145],
 [50.432943

In [146]:
df1['lat_long'] = list_coordinat
df1

<ipython-input-146-19a549d713ad>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['lat_long'] = list_coordinat


,district,metro,area,rooms,floor,gk,price,adress,lat_long,repair,metro_dist
0,1.19,1,85.0,2,4,0,30200,"\nул. Липкивского Василия (Урицкого), 18, Киев\n","[50.4339983, 30.4843094]",0,3
1,1.22,3,40.0,1,4,0,10900,"\nул. Богатырская, 6а, Киев\n","[50.5201598, 30.4849631]",0,1
2,1.22,1,40.0,1,4,1,9500,"\nул. Калнишевского Петра (Майорова М.), 8, Ки...","[50.5222776, 30.4663735]",0,1
3,1.29,3,50.0,2,4,0,25000,"\nул. Боричев Ток, 8, Киев\n","[50.4594295, 30.5215163]",1,1
4,1.12,1,97.0,3,23,1,65900,"\nул. Днепровская наб., 14б, Киев\n","[50.4017441, 30.614014]",0,1
...,...,...,...,...,...,...,...,...,...,...,...
7718,1.19,3,70.0,2,4,0,22000,"\nул. Никольско-Слободская, 3а, Киев\n","[50.4551297, 30.5861282]",0,1
7719,1.19,1,41.0,1,2,1,14000,"\nул. Регенераторная, 4 корпус 2, Киев\n","[50.4375504, 30.6213166]",0,1
7720,2.87,1,170.0,4,4,1,68600,"\nДружбы Народов бульв., 14-16, Киев\n","[50.413621, 30.5372004]",0,1
7721,1.74,3,50.0,2,4,1,27500,"\nул. Златоустовская, 30, Киев\n","[50.4505902, 30.4879963]",0,2


In [24]:
df1[df['price'] == 299000]

<ipython-input-24-a7e1c87d3e00>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1[df['price'] == 299000]


,district,metro,area,rooms,floor,gk,price,adress
2307,2.87,1,341.0,4,4,0,299000,"\nул. Мичурина, 56/2, Киев\n"
2309,2.87,1,341.5,3,4,0,299000,"\nул. Мичурина, 56/2, Киев\n"


In [ ]:
df1

In [124]:
df1.to_csv('rent100riel_coordin_repair_new_dist.csv', encoding='utf-8', index=False)